In [1]:
import librosa
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [2]:
# definesc functiile necesare citirii si prelucrarii datelor de intrare
# https://medium.com/comet-ml/applyingmachinelearningtoaudioanalysis-utm-source-kdnuggets11-19-e160b069e88

# C-UL SE CAUTA DIN 10 IN 10
# un interval bun: 10^(-1), 10^(-2), 1, 10, 10^2

def read_sounds(files, input_file):
    sounds = []
    for i in range(len(files)):
        x, sample_rate = librosa.load(input_file + '/' + input_file + '/' + files[i], res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=128).T, axis=0)
        sounds.append(mfccs)
    return np.array(sounds)

def read_data(input_file):
    files, labels = np.loadtxt(input_file + '.txt', dtype='str', delimiter=',', unpack=True)
    return read_sounds(files, input_file), np.array(labels.astype(int))

def read_test(input_file='test'):
    test_files = np.loadtxt(input_file + '.txt', dtype='str', unpack=True)
    return test_files, read_sounds(test_files, input_file)

In [3]:
# citesc datele de intrare

train_data, train_labels = read_data('train')
val_data, val_labels = read_data('validation')
test_files, test_data = read_test()

In [8]:
# declar modelul folosit si il antrenez pe datele de intrare
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

model = SVC(C=100, kernel='linear')
model.fit(train_data, train_labels)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [9]:
# scorul pe datele de antrenare

model.score(train_data, train_labels)

0.733125

In [10]:
# scorul pe datele de validare

model.score(val_data, val_labels)

0.677

In [24]:
# matricea de confuzie pentru datele de validare

predicted_val_labels = model.predict(val_data)
confusion_matrix_val = confusion_matrix(val_labels, predicted_val_labels)
print(confusion_matrix_val)

[[328 144]
 [154 374]]


In [25]:
# sparg matricea de validare

true_negative = confusion_matrix_val[0][0]
false_positive = confusion_matrix_val[0][1]
false_negative = confusion_matrix_val[1][0]
true_positive = confusion_matrix_val[1][1]

In [27]:
# calculez precizia

precision_score = true_positive / (true_positive + false_positive)
print(precision_score)

0.722007722007722


In [28]:
# calculez recall-ul

recall_score = true_positive / (true_positive + false_negative)
print(recall_score)

0.7083333333333334


In [1]:
# calculez si salvez predictiile pentru datele de test
# https://stackoverflow.com/questions/36210977/python-numpy-savetxt-header-has-extra-character/36211002

test_labels = model.predict(test_data)
np.savetxt('submission0.txt', np.c_[test_files, test_labels], delimiter=',', header='name,label', comments='', fmt='%s')

NameError: name 'model' is not defined